In [ ]:
import pdfplumber
import re

Hello Word


In [ ]:
def extract_and_read(pdf_path):
    text_by_page = []
    images_by_page = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            #extrai texto da página
            text_by_page.append(page.extract_text())

            #verificar imagem
            #pdfplumber retorna uma lista de dicionario para cada imagem encontrada
            if page.images:
                images_by_page.append(True)
            else:
                images_by_page.append(True)
    return text_by_page, images_by_page

In [ ]:
def extract_resposta(resposta_texto):
    gabarito = {}
    regex = r"(\d+)\s*[-.:]?\s*([A-E])"
    matches = re.findall(regex, answer_key_text)
    
    for match in matches:
        questao_numero = int(match[0])
        resposta = match[1]
        gabarito[questao_numero] = resposta
    return gabarito

In [ ]:
import re

def parse_exam_questions(exam_text_pages, gabarito, exam_color):
    questions_data = []
    full_exam_text = "\n".join(exam_text_pages) # Concatena todo o texto das páginas

    # Regex para encontrar o início de uma questão (ex: "QUESTÃO 1", "1.")
    # Ajuste esta regex para ser mais específica se necessário (ex: números no início da linha)
    question_start_regex = r"(?:QUESTÃO\s*)?(\d+)\."

    # Encontra todas as posições dos inícios das questões
    matches = list(re.finditer(question_start_regex, full_exam_text))

    for i, match in enumerate(matches):
        question_number = int(match.group(1))
        start_index = match.end() # Onde a questão realmente começa após o número

        # Determina o fim da questão: ou o início da próxima questão ou o fim do documento
        end_index = matches[i+1].start() if i + 1 < len(matches) else len(full_exam_text)

        # Extrai o texto bruto da questão
        question_raw_text = full_exam_text[start_index:end_index].strip()

        # Inferir se possui imagem (muito simplificado, idealmente requer análise de coordenadas)
        # Por enquanto, vamos assumir 'False' e você pode integrar a lógica de image_presence aqui
        possui_imagem = False # Lógica real virá da análise do pdfplumber

        # Extrair enunciado: Remover as opções A, B, C, D, E do final do texto
        # Esta regex encontra as opções e o texto antes delas é o enunciado
        enunciado_regex = r"(.*)\n\s*A\)\s*.*\n\s*B\)\s*.*\n\s*C\)\s*.*\n\s*D\)\s*.*\n\s*E\)\s*.*"
        enunciado_match = re.search(enunciado_regex, question_raw_text, re.DOTALL)
        enunciado = enunciado_match.group(1).strip() if enunciado_match else question_raw_text.split('\n')[0] # Pega a primeira linha se não encontrar as opções

        # Adiciona os dados da questão à lista
        questions_data.append({
            "Número da Questão": question_number,
            "Enunciado da Questão": enunciado,
            "possui_imagem": possui_imagem, # Isso precisa ser preenchido com a lógica real
            "Gabarito da Questão": gabarito.get(question_number, "N/A"), # Busca no gabarito
            "Cor da Prova": exam_color
        })
    return questions_data

# Exemplo de uso (simulado):
# exam_pages_text_example = [
#     """
#     QUESTÃO 1. Texto do enunciado da questão 1.
#     A) Opção A
#     B) Opção B
#     C) Opção C
#     D) Opção D
#     E) Opção E

#     QUESTÃO 2. Este é o enunciado da questão 2. Pode ter várias linhas.
#     A) Opção A2
#     B) Opção B2
#     C) Opção C2
#     D) Opção D2
#     E) Opção E2
#     """
# ]
# simulated_gabarito = {1: 'C', 2: 'A'}
# simulated_exam_color = "Amarela"
# parsed_questions = parse_exam_questions(exam_pages_text_example, simulated_gabarito, simulated_exam_color)
# for q in parsed_questions:
#     print(q)